In [ ]:
using Pkg;Pkg.status()

In [ ]:
using TensorQEC
using TensorQEC.Yao
using TensorQEC.OMEinsum
using Random

## Circuit-level Quanutm Error Correction Decoding Problem
### Load data
The quantum circuits and the corresponding detector error model is placed under the `data` folder. Here we load the syndrome measurement circuit of code distance `d=3`, error correction cycle `r=3` surface code and the corresponding detector error model. It is from Ref: xxx

In [ ]:
# stim file stores the circuit and the error model
qc = parse_stim_file(joinpath(@__DIR__, "data", "surface_code_d=3_r=3.stim"), 26);
vizcircuit(qc)

- The circuit first measure and reset the ancilla qubits to state 0. The unmeasured lines represents the data qubits.
- After each gate, depolarizing error is added.
- Intermediate measurement outcomes are stored into the recorder, annotated by `rec[k]`
- Detectors checks the syndromes, it checks the recorder and computes the XOR of the annotated records.

In [ ]:
# dem file stores the XXX, which can be used to sample the errors
dem = TensorQEC.parse_dem_file(joinpath(@__DIR__, "data", "surface_code_d=3_r=3.dem"))

### Generate the tensor network
Now we can generate the corresponding tensor network with `compile` function. `TNMMAP` is a tensor network based marginal maximum a posteriori (MMAP) decoder, which finds the most probable logical sector after marginalizing out the error pattern on qubits. `TreeSA()` is the optimizer for optimizing the tensor network contraction order. `true` means that we want to factorize the tensors to rank-3 tensors to avoid large tensors. (we should do this implicitly)

In [ ]:
compiled_decoder = compile(TNMMAP(TreeSA(), true), dem)
compiled_decoder.code

In [ ]:
contraction_complexity(compiled_decoder.code,uniformsize(compiled_decoder.code, 2))

## Compute the probability of different logical sectors by tensor network contraction

In [ ]:
# Randomly generate an error pattern and measure the syndrome.  ## Should generated from the circuit?
Random.seed!(1234)
ep = random_error_qubits(IndependentFlipError(dem.error_rates))
syndrome = syndrome_extraction(ep, compiled_decoder.tanner)

# Update the syndrome and compute the probability of different logical sectors by tensor network contraction.
TensorQEC.update_syndrome!(compiled_decoder, syndrome)
compiled_decoder.code(compiled_decoder.tensors...)

### Another hard example of [[144,12,12]] BB Code.
This file comes from https://github.com/quantumlib/tesseract-decoder/tree/main/testdata/bivariatebicyclecodes

In [ ]:
dem = TensorQEC.parse_dem_file(joinpath(@__DIR__, "data", "r=12,d=12,p=0.001,noise=si1000,c=bivariate_bicycle_X,nkd=[[144,12,12]],q=288,iscolored=True,A_poly=x^3+y+y^2,B_poly=y^3+x+x^2.dem"))
ct = compile(TNMMAP(TensorQEC.NoOptimizer(), true), dem); # Here we use the NoOptimizer to avoid any optimization. Since the code is too large, the default optimizer will be too slow.
length(ct.code.ixs)

In [ ]:
contraction_complexity(ct.code,uniformsize(ct.code, 2))